# Setups

## Import

In [1]:
import numpy as np
import pandas as pd
from itertools import chain

import torch
import torch.nn as nn
import torch.nn.functional as F

from preprocess import *

## Load Data

In [2]:
%%time
df = Preprocess().preprocess()

Wall time: 50.6 s


# Setups

## Hyperparams

In [3]:
item_ids = list(chain(*df['assessmentItemID'].values))
test_ids = list(chain(*df['testId'].values))

In [90]:
seq_len = 256
num_items = len(set(item_ids))
num_tests = len(set(test_ids))
num_feats = len(df.columns)

## Modeling

### Chunk

In [5]:
split = lambda data, split_size: np.split(data[:split_size], split_size // seq_len)
pad = lambda row, pad_len: F.pad(torch.tensor(row), (0, pad_len), value=-1)

In [70]:
df.columns

Index(['assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'Elapsed',
       'testConsecutive'],
      dtype='object')

In [71]:
chunked_data = []
count = 0
for row in df.values:
    
    row = tuple(map(np.array, row))
    
    if len(row[0]) >= seq_len:
        
        split_size = len(row[0]) - len(row[0])%seq_len        
        # TODO: simple and clean. Currently,,, only simple
        chunked_data.extend(list(zip(*map(split, row, [split_size]*num_feats))))
        
    elif len(row[0]) < seq_len:
        
        pad_len = seq_len - len(row[0])
        chunked_data.append(tuple(pad(row, pad_len).numpy()))
        
    else:
        pass
        
chunked_data = pd.Series(chunked_data)

### DataLoader

In [83]:
import torch
from torch.utils.data import Dataset, DataLoader

In [84]:
class IscreamDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        return tuple(torch.LongTensor(feat) for feat in self.data[index])
    
    def __len__(self):
        return len(self.data)

In [91]:
def collate_fn_short(batch):
    
    batchify = lambda idx: torch.cat([b[idx].unsqueeze(1) for b in batch], dim=1)
    return [batchify(i) for i in range(num_feats)]

In [85]:
pin_memory = False

trainset = IscreamDataset(chunked_data)
train_loader = DataLoader(trainset, shuffle=False,
                          batch_size=16,
                          drop_last=True,)
#                           collate_fn=collate_fn_short)

In [86]:
batch = next(iter(train_loader))

### Model

In [92]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [94]:
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable

import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.nn import TransformerDecoder, TransformerDecoderLayer

In [93]:
from src.utils import PositionalEncoding, NoamOpt

ModuleNotFoundError: No module named 'src'

In [ ]:
class TransformerModel(nn.Module):
    
    def __init__(self, intoken, hidden, part_arr, enc_layers, dec_layers, dropout, nheads, ff_model, ts_unique=70):
        super(TransformerModel, self).__init__()
        
        # Encoders' Embeddings
        self.item_emb = nn.Embedding(num_items, hidden)
        self.test_emb = nn.Embedding(num_tests, hidden)
        self.ts_emb = nn.Embedding(intoken, hidden) # TODO:
        self.elapse_emb = nn.Embedding() # TODO:
        self.consec = nn.Embedding() # TODO:
        
        self.src_mask = None
        self.trg_mask = None
        self.memory_mask = None # TODO:
        
        self.pos_enc = PositionalEncoding(hidden, dropout)
        
        # Decoders' Embeddings
        self.ans_emb = nn.Embedding(3, hidden)
        self.pos_dec = PositionalEncoding(hidden, dropout)      
        
        self.fc_out = nn.Linear(hidden, 1)
        
        self.transformer = nn.Transformer(d_model=hidden,
                                         n_head=n_heads,
                                         num_enc_layers=enc_layers,
                                         num_dec_layers=dec_layers,
                                         dim_ff=ff_model,
                                         dropout=dropout,
                                         activation='relu')
        
    def generate_square_subsequent_mask(self, sz, sz1=None):
        
        if sz1 == None:
            mask = torch.triu(torch.ones(sz, sz), 1)
        else:
            mask = torch.triu(torch.ones(sz, sz1), 1)
            
        return mask.masked_fill(mask==1, float('-inf'))

# 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'Elapsed','testConsecutive'
    def forward(self, item, test, ans, ts, elapse, consec):
        
        if self.trg_mask is None or self.trg_mask.size(0) != len(trg):
            self.trg_mask = self.generate_square_subsequent_mask(len(trg)).to(trg.device)
            
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            self.src_mask = self.generate_square_subsequent_mask(len(src)).to(trg.device)
            
        if self.memory_mask is None or self.memory_mask.size(0) != len(trg) or self.memory_mask.size(1) != len(src):
            self.memory_mask = self.generate_square_subsequent_mask(len(trg),len(src)).to(trg.device)
            
        pass